In [1]:
#libreria usada
# pip install PyMuPDF

Note: you may need to restart the kernel to use updated packages.


In [1]:
import fitz
import re
import pandas as pd

In [2]:
#secarga el pdf y se pasa a miniscula 
pdf = '2.pdf'
doc = fitz.open(pdf)
salida = open("2"+".txt","wb") 
for pagina in doc:
    texto = pagina.getText().lower().encode("utf8")
    salida.write(texto)
    salida.write(b"\n-----\n")

salida.close()

Deprecation: 'getText' removed from class 'Page' after v1.19 - use 'get_text'.


In [3]:
# cargamos las lineas del texto y las lineas unidas
with open('2.txt', 'r', encoding="utf8") as f:
    lineas = [linea.split() for linea in f]

lineas_unidas = []
for i in range(len(lineas)):
    lineas_unidas += lineas[i]


In [4]:
# funcion que busca la posicion de la palabra
def finder(x):
    for i in range(len(lineas)):
        try:
            return i,lineas[i].index(x)
        except ValueError:
            pass     

In [5]:
# buscamos la palabra
neighborhood_1 = lineas[finder('glomerular')[0]][finder('glomerular')[1]:finder('glomerular')[1]+4] 
neighborhood_2 = lineas[finder('renal')[0]][finder('renal')[1]:finder('renal')[1]+4] 
if(neighborhood_2!=[]):neighborhood_3 = lineas[finder('fecha')[0]][finder('fecha')[1]:finder('fecha')[1]+4] 


# ----- para riesgo ------
#Establecer las variantes o formas de escribir riesgo    
myDict = {'riesgo':1, 'reisg':1,'risk':1,'reisgo':1,'riak':1,'reisg':1}    

#Determinar posición de las palabras riesgo o similares
posiciones_riesgo = [index for index, word in enumerate(lineas_unidas) if word in myDict]

#Determinar cuales o cuales con las siguiente palabras despues de las palabras riesgo 
posicion_cardio = [lineas_unidas[i] for i in list(map(lambda x: x+1, posiciones_riesgo))]

#"vecinos superiores (4 palabras)" de la palabra cardiovascular que precede a la palabra riesgo o similar.
neighborhood_4 = lineas_unidas[posiciones_riesgo[(posicion_cardio.index("cardiovascular"))]:
                               posiciones_riesgo[(posicion_cardio.index("cardiovascular"))]+4] 
    

neighborhood_5 = lineas[finder('cardiovascular')[0]][finder('cardiovascular')[1]:finder('cardiovascular')[1]+4]   

# encontrar variables
frase_1 = " ".join(neighborhood_1) #Encontrar el número
frase_2 = " ".join(neighborhood_2)
frase_3 = " ".join(neighborhood_3)
frase_4 = " ".join(neighborhood_4)
frase_5 = " ".join(neighborhood_5)

#expresion regular para tomar lo que se quiere unicamente
s1 = [float(s) for s in re.findall(r'-?\d+\.?\d*',frase_1)]
s2 = [s for s in re.findall(r'[^renal ][^por ][ ^a-z]*',frase_2)]
s3 = [s for s in re.findall(r'\d{2}/\d{2}/\d{4}',frase_3)]
s4 = [s for s in re.findall(r'-?\d+\%*',frase_4)]
s5 = [s for s in re.findall(r'[^por ][a-z][ ^a-z]*',frase_5)]


print(s1)
print(s2)
print(s3)
print(s4)
print(s5[1])

[34.0]
['ckd epi']
['26/07/2019']
['15%']
framingham


In [6]:
data = {
    "Tasa de filtración glomerular": s1[0],
    "Insuficiencia renal": s2[0],
    "Fecha de identificación de insuficiencia renal":s3[0],
    "Riesgo cardiovascular": s4[0], 
    "Método de cálculo":s5[1]
}

df= pd.DataFrame(list(data.items()),
                  columns = ['Variable','Valor'])
print(df)

df.to_excel('df.xlsx')

                                         Variable       Valor
0                   Tasa de filtración glomerular        34.0
1                             Insuficiencia renal     ckd epi
2  Fecha de identificación de insuficiencia renal  26/07/2019
3                           Riesgo cardiovascular         15%
4                               Método de cálculo  framingham
